In [4]:
from abc import ABC, abstractmethod

class Reality():


    def __init__(
            self, person_list=[], broker_list=[], index_list=[], starting_time_period=0):
        self.person_list = person_list
        self.broker_list = broker_list
        self.index_list = index_list
        self.time_period = starting_time_period
        self.person_data = {}
        self.index_data = {}

    def execute_person_period(self, person, income_change=0, 
                              acc_list=None, acc_index_dict=None):
        person.change_monthly_income(income_change=income_change)
        person.receive_monthly_income()
        for acc in acc_list:
            person.add_account(booker_account=acc)
        for acc in acc_index_dict:
            for index in acc_index_dict[acc]:
                person.add_instrument_to_account(account=acc, index=index)
        person.execute_investing_behaviour()
        for account in person.broker_accs:
            account.calculate_inst_amount_wraper()
        self.save_person_data(person)

    def execute_indexes_period(self):
        for index in self.index_list:
            self.execute_index_period(index)

    def execute_index_period(self, index):
        index.index_calculate_current_price()
        index.calculate_multip()
        self.save_index_data(index=index)

    def execute_people_period(self):
        for person in self.person_list:
            person.execute_person_period(person=person)

    def save_person_data(self, person):
        if not self.person_data[person]:
              self.person_data[person] = {}
        self.person_data[person][self.t] = {}
        self.person_data[person][self.t]['amount'] = person.amount
        self.person_data[person][self.t]['income'] = person.current_monthly_income
        self.person_data[person][self.t]['age'] = person.age
        for account in person.broker_accs:
            self.person_data[person][self.t]['accounts'][account.booker_name] = {}
            self.person_data[person][self.t]['accounts'][account.booker_name]['amount'] = account.amount
            self.person_data[person][self.t]['accounts'][account.booker_name]['instruments'] = account.index_dict
    
    def save_index_data(self, index):
        self.index_data[index.name][self.t] = index.current_price

    def execute_period(self):
        self.t += 1
        self.execute_indexes_period()
        self.execute_people_period()


class Person():

    def __init__(
            self, name, starting_age, starting_money, current_monthly_income, reality, booker_acounts=[]):
        self.name = name
        self.age = starting_age
        self.money = starting_money
        self.current_monthly_income = current_monthly_income 
        self.broker_accs = booker_acounts
        self.investing_behaviour = {}
        self.reality = reality

    def add_account(self, broker_name):
        if broker_name not in self.reality.broker_list:
            return("Broker does not exist in this reality")
        account = Account(owner=Person.name, booker_name=broker_name)
        self.broker_accs.append(account)

    def delete_account(self, account):
        self.broker_accs.remove(account)

    def add_index_to_account(self, account, index):
        if account not in self.booker_acounts:
            return('Account does not belong to the person')
        self.broker_accs[account].index_dict[index.name] = 0

    def add_money_to_account(self, account, money):
        if account not in self.booker_accounts:
            return('Account does not belong to this person')
        self.broker_accs[account].account_amount += money

    def take_money_from_account(self, account, money):
        if account not in self.booker_accounts:
            return('Account does not belong to this person')
        self.broker_accs[account].account_amount -= money
        self.money += money

    def add_money_to_index(self, account, index, money):
        account.transfer_money_to_instrument(index=index.name, money=money)
        
    def receive_monthly_income(self):
        self.money = self.money + self.current_monthly_income

    def change_monthly_income(self, income_change):
        self.current_monthly_income = (self.current_monthly_income  
                                       + income_change)
                
    def execute_investing_behaviour(self, behaviour_dict):
        for account in behaviour_dict:
            account_total = sum(behaviour_dict[account].values())
            self.add_money_to_account(account=account_total,
                                       money=account_total
                                       *self.current_monthly_income)
            for index in account:
                self.add_money_to_index(account=account, instrument=index, 
                                        money=(
                                            self.current_monthly_income
                                            *behaviour_dict[account][index]))
        
class Account():

    def __init__(self, owner, booker_name, starting_index_dict={}, starting_amount=0):
        self.booker_name = booker_name
        self.owner = owner
        self.index_dict = starting_index_dict
        self.account_amount  = starting_amount

    def add_instrument(self, index):
        self.index_dict[index.name] = 0
    
    def delete_instrument(self, index):
        if self.index_dict[index.name] != 0:
            return('can delete account, there are still money in it')
        del self.index_dict[index.name]

    def calculate_instrument_amount(self, index_name):
        if index_name not in self.index_dict:
            print('Index is not bought in current account')
            return
        self.index_dict[index_name] = self.index_dict[index_name] * 2.4

    def calculate_inst_amount_wraper(self):
        for index_name in self.index_dict:
            self.calculate_index_amount(index_name=index_name)

    def transfer_money_to_instrument(self, instrument, money):
        if money > self.account_amount:
            return('not enough money for the transaction')
        self.account_amount = self.account_amount - money
        self.index_dict[instrument.name] += money


class  Instrument(ABC):

    def __init__(self, name, type_):
        self.name = name
        self.type = type_

    @abstractmethod
    def calculate_multip(self):
        pass

    @abstractmethod
    def execute_time_period(self):
        pass


class Index(Instrument):

    def __init__(
            self, name, type_='index', price_ts=[], current_price=None, previous_price=None):
        super().__init__(type_, name)
        self.current_price = current_price
        self.previous_price = previous_price
        self.multip = 0
        self.price_ts = price_ts

    def calculate_current_price(self, t):
        self.previous_price = self.current_price 
        self.current_price 
        self.price_ts[t] = self.current_price
        pass

    def calculate_multip(self):
        (self.current_price - self.previous_price) // self.previous_price

    def execute_time_period(self):
        self.calculate_current_price()
        self.calculate_multip()




 

In [5]:
a = Index(name='aa')

In [13]:
reality = Reality()
person = Person(name='ja', starting_age=25, starting_money=0, reality=reality, current_monthly_income=1250)

In [14]:
reality.broker_list.append('kb')

In [16]:
person.reality.broker_list

['kb']

In [10]:
person.reality.broker_list

In [4]:
ind_1 = Index(name='SAP', type_='etf')
acc = Account()